# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

# Imports 

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Cruise').getOrCreate()

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled',True)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows',5)

# Data

In [ ]:
data = spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [ ]:
data

Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew
Journey,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55
Quest,Azamara,6,30.276999999999997,6.94,5.94,3.55,42.64,3.55
Celebration,Carnival,26,47.262,14.86,7.22,7.43,31.8,6.7
Conquest,Carnival,11,110.0,29.74,9.53,14.88,36.99,19.1
Destiny,Carnival,17,101.353,26.42,8.92,13.21,38.36,10.0


Note: The hint that ships differ in acceptable crew counts hints in the importances of the cruise_line variable

# Feature Engineering

In [ ]:
# Tokenize the string column

from pyspark.ml.feature import StringIndexer

In [ ]:
string_indexer = StringIndexer(inputCol='Cruise_line',outputCol='Cruise_index')
model = string_indexer.fit(data)

In [ ]:
data2 = model.transform(data).drop('Ship_name','Cruise_line')
data2

Age,Tonnage,passengers,length,cabins,passenger_density,crew,Cruise_index
6,30.276999999999997,6.94,5.94,3.55,42.64,3.55,16.0
6,30.276999999999997,6.94,5.94,3.55,42.64,3.55,16.0
26,47.262,14.86,7.22,7.43,31.8,6.7,1.0
11,110.0,29.74,9.53,14.88,36.99,19.1,1.0
17,101.353,26.42,8.92,13.21,38.36,10.0,1.0


In [ ]:
# Transform to features/labels dataframe 

from pyspark.ml.feature import VectorAssembler


In [ ]:
data2.columns

['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_index']

In [ ]:
assembler = VectorAssembler(
    inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density','Cruise_index'],
    outputCol='features')


In [ ]:
output = assembler.transform(data2)

In [ ]:
data3 = output.select("features","crew").withColumnRenamed("crew","label")

In [ ]:
data3

features,label
"[6.0,30.276999999...",3.55
"[6.0,30.276999999...",3.55
"[26.0,47.262,14.8...",6.7
"[11.0,110.0,29.74...",19.1
"[17.0,101.353,26....",10.0


In [ ]:
train, test = data3.randomSplit([0.7,0.3])

# Model Fitting

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
model = lr.fit(train)

# Model Evaluation

In [ ]:
evaluations = model.evaluate(test)

In [ ]:
evaluations.r2

0.9621022834026965

In [ ]:
evaluations.meanSquaredError

0.5234480975485